# Crowd dynamics simulation — a multi-agent system based on CA

# Tasks

## Neighbourhood
Fill in the missing part of the *\_\_init\_\_* method in the *Board* class. Prepare two versions: one with Moore and one with von Neumann neighbourhood. Do not initialise neighbourhoods for border cells.
## Static potential field (2 points)

### Implement method *calculateField* in *Board* class:
* Create a list of points (*toCheck*) for which the static field should be recalculated (at the beginning all *staticFields* are equal to 100000). 
    * For all exits (points with *pointType == 2*) change the *staticField* to 0. 
    * Add all exits' neighbours to the *toCheck* list.
* Until the list *toCheck* is empty:
    * chech whether the first element of the list changed its *staticField* (call *calcStaticField*).
    * If so, add all its neighbours to the *toCheck* list.
    * Remove the first element from the list.

### Implement method *calcStaticField* in *Point* class:
* Find the smallest *staticField* in the cell's neighbourhood.
* If the cell's *staticField* is greater than the found value + 1, change the *staticField* to the found value + 1 and return *True*, otherwise do not change anything and return *False*.

### Implement *randomBoard* method:
* Add some walls and pedestrians, add at least one exit.
* Try to do this randomly.

## Naive Implementation ( 2 points)

### Implement *move* method:
* If the cell represents a pedestrian (*pointType == 3*), move the pedestrian to the neighbouring cell with the lowest *staticField*.

Run the simulation and observe what is happening. What's wrong? Try to find why some artefacts can be observed.

## Improvements (2 points)
Two main reasons responsible for the errors are:
* No "exit" mechanism. After reaching the exit, the agent should be removed.
* No cells synchronisation. One should note that agents moving down and right can reach the destination in one iteration.
* To fix this issue add a boolean value *isBlocked = false* to the *Point* class. After the agent moves, the *isBlocked* value of the occupied cell should change to *True*. Remember to "unblock" all cells at the beginning of each iteration. 

Correct the errors and create a complex board with walls, many pedestrians and at least two exits.





In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import itertools
from IPython.display import HTML

In [2]:
class Point:
    def __init__(self):
        self.neighbours = []
    # 0 - floor, 1 - wall, 2 - exit, 3 - pedestrian
        self.pointType = 0
        self.staticField = 100000
        self.isBlocked = False

    def clear(self):
        self.staticField = 100000


    def calcStaticField(self):
        # TODO
        neigh = list(map(lambda x: x.staticField, self.neighbours))
        if len(neigh) > 0:
            neighbourMin = min(neigh)
            if self.staticField > neighbourMin+1:
                self.staticField = neighbourMin+1
                return True
        return False

    def move(self):
        # TODO
        if self.pointType == 3 and self.isBlocked == False:
            # Pedestrian
            freeNeighbours = list(filter(lambda x: (x.isBlocked == False and (x.pointType==0 or x.pointType==2)), self.neighbours))
            if len(freeNeighbours) > 0:
                pos = freeNeighbours[0]
                minValue = freeNeighbours[0].staticField
                for i in freeNeighbours:
                    if i.staticField < minValue:
                        minValue = i.staticField
                        pos = i
                if pos.pointType == 2:
                    self.pointType = 0
                    pos.isBlocked = True
                else:
                    self.pointType = 0
                    pos.pointType = 3
                    pos.isBlocked = True
            
               
                    
        return


    def getColour(self):
        if self.pointType == 0:
            # Floor
            return 0
        elif self.pointType == 1:
            # Wall
            return 255
        elif self.pointType == 2:
            # Exit
            return 64
        else:
            # Pedestrian
            return 128
        


In [3]:
class Board:
    
    def __init__(self, xSize, ySize):
        self.points = []
        for i in range(0, xSize):
            row = []
            for j in range(0, ySize):
                row.append(Point())
            self.points.append(row)
                
        for i in range(1, xSize-1):
            for j in range(1, ySize-1):
                # TODO initialize neighbourhood here
                # Moore
                tmpList = [self.points[i-1][j-1], self.points[i-1][j], self.points[i-1][j+1],
                          self.points[i][j-1], self.points[i][j+1], self.points[i+1][j-1], self.points[i+1][j], self.points[i+1][j+1]]
    
                self.points[i][j].neighbours = tmpList
                                                    
    def randomlyAddPedestrian(self):
        self.points[10][10].pointType = 3
        self.points[20][20].pointType = 3
        self.points[3][1].pointType = 3


    

        
    def iteration(self):
        for i in range(1,len(self.points)-1):
            for j in range(1, len(self.points[0])-1):
                self.points[i][j].isBlocked = False
                
        for i in range(1,len(self.points)-1):
            for j in range(1, len(self.points[0])-1):
                self.points[i][j].move()

    
    def clear(self):
        for i in range(1,len(self.points)-1):
            for j in range(1, len(self.points[0])-1):
                self.points[i][j].clear()
        self.calculateField()
    
    def redraw(self):
        # Returns numpy array of colours
        fig = np.zeros((len(self.points),len(self.points[0])))
        for i in range(0,len(self.points)):
            for j in range(0, len(self.points[0])):
                fig[i][j] = self.points[i][j].getColour()
        return fig
    
    def getFieldMap(self):
        # Returns numpy array of fieldValues
        fig = np.zeros((len(self.points),len(self.points[0])))
        for i in range(0,len(self.points)):
            for j in range(0, len(self.points[0])):
                fig[i][j] = self.points[i][j].staticField
        return fig
    
    def calculateField(self):
        # TODO 
        toCheck = []
        for i in range(1,len(self.points)-1):
            for j in range(1, len(self.points[0])-1):
                if self.points[i][j].pointType == 2:
                    self.points[i][j].staticField = 0
                    toCheck.extend(self.points[i][j].neighbours)
        while len(toCheck) > 0:
            if toCheck[0].calcStaticField() == True:
                toCheck.extend(toCheck[0].neighbours)
            toCheck.pop(0)
                
            

In [4]:
def update(frameNum, board, im):
    if(frameNum == 0):
        im.set_data(board.redraw()) 
#         print(board.getFieldMap())


    else:   
        board.iteration()
        im.set_data(board.redraw()) 
    return im,

In [5]:
%%capture
# Parameters
steps = 100
# Config
fig, ax = plt.subplots(figsize=(10,10))
plt.axis('off')


In [6]:
#Run Simulation
import random 
board = Board(30,30)
board.randomlyAddPedestrian()

for i in range(0,len(board.points)):
    board.points[0][i].pointType = 1
    board.points[-1][i].pointType = 1
    board.points[i][0].pointType = 1
    board.points[i][-1].pointType = 1
    board.points[i][15].pointType = 1



for i in range (1,len(board.points)-1):
    for j in range (1,len(board.points)-1):
        randValue = random.randint(0,100)
        if randValue < 10:
            board.points[i][j].pointType = 3
            
board.points[2][2].pointType=2
board.points[-10][-5].pointType=2
board.points[3][3].pointType=1
board.points[7][15].pointType=0
board.points[3][4].pointType=1
board.points[4][3].pointType=1
board.points[4][4].pointType=1






board.clear()
# plt.figure()
# plt.imshow(board.getFieldMap(),vmin=0, vmax=255)
im = ax.imshow(board.redraw(),  vmin=0, vmax=255)
ani = animation.FuncAnimation(fig, update, frames = steps, fargs = [board, im])
HTML(ani.to_jshtml())


In [7]:
print(board.getFieldMap())

[[1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05
  1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05
  1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05 1.0e+05
  1.0e+05 1.0e+05 1.0e+05]
 [1.0e+05 1.0e+00 1.0e+00 1.0e+00 2.0e+00 3.0e+00 4.0e+00 5.0e+00 6.0e+00
  7.0e+00 8.0e+00 9.0e+00 1.0e+01 1.1e+01 1.2e+01 1.3e+01 1.4e+01 1.5e+01
  1.6e+01 1.7e+01 1.8e+01 1.9e+01 1.9e+01 1.9e+01 1.9e+01 1.9e+01 1.9e+01
  1.9e+01 1.9e+01 1.0e+05]
 [1.0e+05 1.0e+00 0.0e+00 1.0e+00 2.0e+00 3.0e+00 4.0e+00 5.0e+00 6.0e+00
  7.0e+00 8.0e+00 9.0e+00 1.0e+01 1.1e+01 1.2e+01 1.3e+01 1.4e+01 1.5e+01
  1.6e+01 1.7e+01 1.8e+01 1.8e+01 1.8e+01 1.8e+01 1.8e+01 1.8e+01 1.8e+01
  1.8e+01 1.8e+01 1.0e+05]
 [1.0e+05 1.0e+00 1.0e+00 1.0e+00 2.0e+00 3.0e+00 4.0e+00 5.0e+00 6.0e+00
  7.0e+00 8.0e+00 9.0e+00 1.0e+01 1.1e+01 1.2e+01 1.3e+01 1.4e+01 1.5e+01
  1.6e+01 1.7e+01 1.7e+01 1.7e+01 1.7e+01 1.7e+01 1.7e+01 1.7e+01 1.7e+01
  1.7e+01 1.7e+01 1.0e+05]
 [1.